In [1]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import os
os.chdir("/content/drive/MyDrive/NLP/lab_1")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/NLP/lab_1'

# Lab 1. Tehnici de bază în prelucrarea textelor

## Regex

Expresiile regulate reprezintă un șir de caractere care definesc un șablon⁠ de căutare.

Ele sunt utile pentru căutarea anumitor șabloane în text și de asemenea, pentru normalizarea textelor - https://www.w3schools.com/python/python_regex.asp


In [ ]:
import re

text = """
Praise for The Rain in Portugal
 
“Nothing in Billy Collins’s twelfth book . . . is exactly what readers might expect, and that’s the charm of this collection.”—The Washington Post
 
“This new collection shows [Collins] at his finest. . . . Certain to please his large readership and a good place for readers new to Collins to begin.”—Library Journal. 
 
“Disarmingly playful and wistfully candid.”—Booklist
Buy new:$38.65
No Import Fees Deposit & $13.01 Shipping to Romania Details -12.3.
"""

Exemplu de utilizare: utilizând metoda `re.sub` ștergem toate caracterele diferite de literele mari și mici ale alfabetului englez, apoi normalizăm toate secvențele de caractere de tip spațiu consecutive la un singur spațiu.

In [ ]:
cleaned_text = re.sub("[^A-Za-z]", " ", text)
cleaned_text = re.sub("\s+", " ", cleaned_text)
print(cleaned_text)

Pentru testarea pattern-urilor putem folosi https://regex101.com/.

### Funcția `finditer`

Această funcție găsește un pattern într-un șir de caractere și returnează un iterator ce generează obiecte de tip Match cu toate potrivirile.

In [ ]:
import re

s = 'Readability counts.'
pattern = r'[aeoui]'

matches = re.finditer(pattern, s)
for match in matches:
    print(match)

Exemplu: căutăm toate numerele float sau int, împreună cu pozițiile și valorile lor. Aici folosim metode `compile()` pentru a compila expresia regulată sub forma de string într-un pattern de tip regex.

In [ ]:
pattern = re.compile("[+-]?(\d+\.)?\d+")
for match in pattern.finditer(text):
    print(match, "--> valoarea căutată începe de la caracterul nr.", match.start(), ", și este ", match.group())

## Encodings

Codificarea (encoding-ul) unui text poate varia, în funcție de limbă și este un element foarte mportant când lucrăm cu texte. 

Python foloseste standardul 'utf-8' pentru limba română, și nu numai. 

Următorul exemplu este preluat dintr-o subtitrare (.srt) din limba rusă, dar nu este encodat in utf-8. Așadar dacă vom încerca să îl citim fără să specificăm tipul de encoding, vom primi următoarea eroare:

In [ ]:
with open('encoded_text.txt', "r") as fin:
    content = fin.read()
    print(content)

Putem detecta encoding-ul folosit cu librăria `chardet`:

In [ ]:
! pip install chardet

In [ ]:
import chardet

with open('encoded_text.txt', "rb") as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    extracted_encoding = result['encoding']
    print("Encoding-ul acestui fișier este: ", extracted_encoding)

Cu encoding-ul potrivit, acum fișierul se poate citi:

In [ ]:
with open('encoded_text.txt', "r", encoding="windows-1251") as fin:
    content = fin.read()
    print(content)

Putem, dacă vrem, să salvam conținutul în format utf-8, deoarece acest format este default pentru python și nu mai trebie specificat la deschidere:

In [ ]:
with open('encoded_text.txt', 'r', encoding=extracted_encoding) as fin:
    content = fin.read()
with open('utf8_text.txt', 'w', encoding='utf-8') as fout:
    fout.write(content)

In [ ]:
with open('utf8_text.txt', "r") as fin:
    content = fin.read()
    print(content)

## Non-standard files (PDF, Word, etc.)

Putem citi texte din documente word folosind librăria `doc2txt`.

In [ ]:
!pip install docx2txt

In [ ]:
import docx2txt
my_text = docx2txt.process("soup.docx")
print(my_text)

Putem citi pdf-uri care sunt salvate ca texte (nu poze), de exemplu, cu librăria `pdfplumber`:

In [ ]:
! pip install pdfplumber

In [ ]:
import pdfplumber
with pdfplumber.open('soup.pdf') as pdf:
    for page in pdf.pages:
        print(page.extract_text())

## Web scraping

Scraping-ul se referă la o mulțime de metode prin care putem descărca date nestructurate din mediul web. Pe noi ne interesează datele text, pe care după preluarea din mediul online le putem procesa și stoca într-o formă structurată.

Ca prim exemplu de scraping vom incerca următorul task: pornind de la site-ul de programare competitiva "infoarena.ro" dorim pentru un utilizator sa descarcam informatii despre toate submisiile efectuate de acesta.

Exemplu pagină de submisii: https://www.infoarena.ro/monitor?user=iordache.bogdan

Pentru a realiza un request care să întoarca conținutul paginii putem folosi librăria `requests`:

In [ ]:
! pip install requests

In [ ]:
import requests

def get_submissions_page(user):
    return requests.get(f"https://www.infoarena.ro/monitor?user={user}")

In [ ]:
html = get_submissions_page("iordache.bogdan").content

Observăm că folosind metoda de mai sus putem descarca întreg conținutul HTML al paginii. Pentru a extrage informații utile trebuie să parsam acest conținut. Pentru aceasta vom folosi biblioteca [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/):

In [ ]:
import bs4

def parse_html(html):
    return bs4.BeautifulSoup(html, "html.parser")

Având conținutul parsat, putem determina acum câte submisii are în total acest utilizator:

In [ ]:
import re

soup = parse_html(html)

# cautam un span care are clasa "count", in acest span se afla numarul de submisii
submission_count_text = soup.find("span", class_="count").text
print(submission_count_text)


Pentru a extrage doar numărul din această înșiruire de caractere ne putem folosi de regex:

In [ ]:
submission_count = int(re.search(r"\d+", submission_count_text).group())
print(submission_count)

Observăm că aceste submisii sunt împărtite în mai multe pagini (paginarea rezultatelor). 

De asemenea, link-ul următor: https://www.infoarena.ro/monitor?user=iordache.bogdan&display_entries=250&first_entry=100 ne returnează 250 de submisii, incepând cu submisia cu numarul 100. 

Putem modifica metoda `get_submissions_page` astfel:

In [ ]:
def get_submissions_page(user, display_entries=None, first_entry=None):
    req_string = f"https://www.infoarena.ro/monitor?user={user}"
    if display_entries is not None:
        req_string += f"&display_entries={display_entries}"
    if first_entry is not None:
        req_string += f"&first_entry={first_entry}"

    return requests.get(req_string)

Și putem implementa o funcție care returnează informații despre toate submisiile unui utilizator:

In [ ]:
from tqdm import tqdm
import pandas as pd
import pdb

def scrape_submissions(user):
    # determinam numarul total de submisii
    html = get_submissions_page(user).content
    soup = parse_html(html)
    submission_count_text = soup.find("span", class_="count").text
    submission_count = int(re.search(r"\d+", submission_count_text).group())

    # vom salva in acest dictionar datele despre submisiile extrase, structura aceasta
    # ne va ajuta ulterior sa construim un tabel (dataframe) folosind pandas
    d = {
        "id": [],
        "problema": [],
        "url_problema": [],
        "url_sursa": [],
        "data": [],
        "puncte": [],
    }

    # accesam pagini cu submisii in grupuri de 250
    for first_entry in tqdm(range(0, submission_count, 250)):
        html = get_submissions_page(user, display_entries=250, first_entry=first_entry).content
        soup = parse_html(html)

        # selectam toate liniile de tabel (tr)
        lines = soup.select("table.monitor tbody tr")

        for line in lines:
            # selectam celulele de pe aceasta linie
            cells = [cell for cell in line.select("td")]

            # extragem link-urile pentru problema si codul sursa
            try:
                url_problema = cells[2].select_one("a")["href"]
                url_sursa = cells[4].select_one("a")["href"]
            except Exception:  # daca vreun link nu exista ignoram linia
                continue
            
            d["id"].append(cells[0].text)
            d["problema"].append(cells[2].text)
            d["url_problema"].append(url_problema)
            d["url_sursa"].append(url_sursa)
            d["data"].append(cells[5].text)

            try:
                puncte = int(re.search(r"\d+", cells[6].text).group())
            except Exception:
                puncte = 0
            d["puncte"].append(puncte)

    return pd.DataFrame(d)

In [ ]:
df_submissions = scrape_submissions("iordache.bogdan")

In [ ]:
df_submissions.head()

In [ ]:
df_submissions.to_csv("submissions.csv", index=False)

Exemplu scriere/citire fisier JSON:

In [ ]:
import json

vec = [
    {"title": "example_1", "size": 7},
    {"title": "example_2", "size": 3},
    {"title": "example_3", "size": 8},
]

with open("example.json", "w") as f:
    json.dump(vec, f, indent=4)

In [ ]:
with open("example.json", "r") as f:
    vec = json.load(f)
print(vec)

Un alt mod de a face scraping este sa folosim biblioteca pandas pentru a ne extrage tabele html, transformandu-le in DataFrame-uri, pe care le putem manipula foarte usor. Un exemplu util este extragerea sărbătorilor legale romanesti, din anul 2022, de pe https://www.timeanddate.com/.

In [ ]:
! pip install lxml

In [ ]:
import pandas as pd

tables_df = pd.read_html('https://www.timeanddate.com/holidays/romania/2022?hol=1')
df = tables_df[0]

# Il putem curata prin a sterge liniile nule si modifica coloanele de la tuplul "(Date, Date)" -> "Date"
df = df.dropna(axis='index')
df.columns = ['Date', 'Day', 'Name', 'Type']

# Reindexam tabelul
df = df.reset_index(drop="True")

# Afisam primele 5 randuri
df.head()

In [ ]:
# Daca vrem se vedem sarbatorile care se nimeresc in ziua de luni putem face o selecție în dataframe
df_luni = df.loc[df["Day"] == "Monday"]
df_luni

Putem salva rezultatul (la fel ca orice dicționar de python) intr-un json, ca alternativa la DataFrame - acest lucru poate fi util într-o aplicație pentru comunicarea cu front-end-ul.

In [ ]:
import json
json_str = df.to_json(orient='records')
json_result = json.loads(json_str)

with open('holidays.json', 'w', encoding='utf8') as fout:
    json.dump(json_result, fout, indent=4, sort_keys=True, ensure_ascii=False)

Alte biblioteci utile pentru scraping:
 * [scrapy](https://scrapy.org/) (folosit in special pentru web crawling)
 * [selenium](https://selenium-python.readthedocs.io/) (folosit pentru a simula activitatea din browser, utilizat in special in scrierea de teste pentru aplicatii front-end)

## TASK: IMDb scraping

File upload:
https://docs.google.com/forms/d/e/1FAIpQLSdOKefipRl6cwjukN5YIxl7Q64dHoUHk2zqg1aO31U7kieHXQ/viewform?usp=sf_link

1. Pornind de la lista cu cele mai populare 250 de filme de pe IMDb ([https://www.imdb.com/chart/top/](https://www.imdb.com/chart/top/)), identificati pentru toate aceste filme link-ul catre pagina sa de recenzii.

Exemplu: aici se gaseste pagina cu recenzii pentru "The Shawshank Redemption": [https://www.imdb.com/title/tt0111161/reviews](https://www.imdb.com/title/tt0111161/reviews)

In [2]:
import bs4
import requests

def parse_html(html1):
    return bs4.BeautifulSoup(html1, "html.parser")


def get_movies_page(chart):
    return requests.get(chart)

html = get_movies_page("https://www.imdb.com/chart/top/").content
soup = parse_html(html)

titles = soup.findAll("td", attrs="titleColumn")

movies = []

for title in titles:

    # gasim link-ul care ne duce la pagina filmului
    a = title.find("a")
    url = a.attrs['href']

    # link-ul contine id-ul filmului care ne trb pt link-ul de reviews
    id = url.split("/")[2]
    reviews_url = f"https://www.imdb.com/title/{id}/reviews"

    print(f"Movie: {a.text}. Reviews link: {reviews_url}")

    movies.append((a.text, reviews_url))


Movie: Închisoarea îngerilor. Reviews link: https://www.imdb.com/title/tt0111161/reviews
Movie: Nașul. Reviews link: https://www.imdb.com/title/tt0068646/reviews
Movie: Cavalerul negru. Reviews link: https://www.imdb.com/title/tt0468569/reviews
Movie: Nașul: Partea a II-a. Reviews link: https://www.imdb.com/title/tt0071562/reviews
Movie: 12 Oameni mânioşi. Reviews link: https://www.imdb.com/title/tt0050083/reviews
Movie: Lista lui Schindler. Reviews link: https://www.imdb.com/title/tt0108052/reviews
Movie: Stăpânul inelelor: Întoarcerea regelui. Reviews link: https://www.imdb.com/title/tt0167260/reviews
Movie: Pulp Fiction. Reviews link: https://www.imdb.com/title/tt0110912/reviews
Movie: Stăpânul inelelor: Frăția inelului. Reviews link: https://www.imdb.com/title/tt0120737/reviews
Movie: Cel bun, cel rău, cel urât. Reviews link: https://www.imdb.com/title/tt0060196/reviews
Movie: Forrest Gump. Reviews link: https://www.imdb.com/title/tt0109830/reviews
Movie: Fight Club - Sala de lupte

2. Pentru fiecare film colectati date despre recenziile sale (titlu, text, rating, data, utlizator, etc.)

In [3]:
def parse_review_page(page, movie):
    tags = page.findAll("div", attrs={"class": "imdb-user-review"})
    reviews = []
    for tag in tags:
        title = tag.find("a", attrs={"class": "title"}).text
        text = tag.find("div", attrs={"class": "text"}).text

        try:
            rating = tag.find("span", attrs={"class": "rating-other-user-rating"}).find("span").text
        except:
            rating = 0

        data = tag.find("span", attrs={"class": "review-date"}).text
        username = tag.find("span", attrs={"class": "display-name-link"}).find("a").text

        reviews.append({
            "title" : title,
            "movie" : movie,
            "text" : text,
            "rating" : rating,
            "data" : data,
            "username" : username})

    return reviews


reviews = list()

for movie in movies:
    page = get_movies_page(movie[1]).content
    html = parse_html(page)

    reviews.extend(parse_review_page(html, movie[0]))



3. Creati un dataset de recenzii, pentru fiecare recenzie stocati:
 * filmul caruia ii apartine
 * titlul recenziei
 * textul recenziei
 * ratingul
 * data
 * utilizator

 Salvati datasetul intr-un fisier JSON.

In [4]:
import json

with open("reviews.json", "w") as file:
    file.write(json.dumps(reviews))

4. Pe o pagina cu recenzii putem gasi un numar mic de astfel de date. Butonul de "Load more" de la final, cand este apasat, produce un request care returneaza HTML-ul urmatoarelor recenzii. Folosind aceasta logica colectati automat pentru fiecare film un numar mai mare de recenzii.

In [5]:
reviews = list()

for movie in movies:
    page = get_movies_page(movie[1]).content
    html = parse_html(page)

    pages = 3

    while html is not None and pages > 0:
        reviews.extend(parse_review_page(html, movie[0]))

        try:
            button = html.find("div", attrs={"class": "load-more-data"}).attrs['data-key']
        except:
            pass

        page = get_movies_page(f"https://www.imdb.com/title/{movie[0]}/reviews/_ajax?ref_=undefined&paginationKey={button}").content
        html = parse_html(page)

        pages -= 1

with open("reviews2.json", "w") as file:
    file.write(json.dumps(reviews))

